In [1]:
from fastai.vision import *
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
img_path = Path("/home/sloth/Desktop/animeface/animeface-2009/animeface-ruby/out-3")

In [3]:
data_source = ImageList.from_folder(img_path).split_none().label_from_folder()

In [4]:
data_source

LabelLists;

Train: LabelList (48666 items)
x: ImageList
Image (3, 89, 89),Image (3, 116, 116),Image (3, 97, 97),Image (3, 116, 116),Image (3, 128, 128)
y: CategoryList
out-3,out-3,out-3,out-3,out-3
Path: /home/sloth/Desktop/animeface/animeface-2009/animeface-ruby/out-3;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: /home/sloth/Desktop/animeface/animeface-2009/animeface-ruby/out-3;

Test: None

In [5]:
tmsf = get_transforms()

In [6]:
data = data_source.transform(tmsf, size=128).databunch(bs=64).normalize(imagenet_stats)

In [7]:
dataloader = data.train_dl.new(shuffle=False)

In [8]:
learner= cnn_learner(data, models.resnet152)

In [9]:
learner.model = torch.nn.DataParallel(learner.model)
model = learner.model.module

In [10]:
for x, y in model.named_modules():
    print(x, y)

 Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 25

In [12]:
# hook code based on https://towardsdatascience.com/similar-images-recommendations-using-fastai-and-annoy-16d6ceb3b809 

class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m:nn.Module, hook_func:HookFunc, is_forward:bool=True, detach:bool=True):
        self.hook_func,self.detach,self.stored = hook_func,detach,None
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.removed = False

    def hook_fn(self, module:nn.Module, input:Tensors, output:Tensors):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach:
            input  = (o.detach() for o in input ) if is_listy(input ) else input.detach()
            output = (o.detach() for o in output) if is_listy(output) else output.detach()
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()
        
def get_output(module, input_value, output):
    return output.flatten(1)

def get_input(module, input_value, output):
    return list(input_value)[0]

def get_named_module_from_model(model, name):
    for n, m in model.named_modules():
        if n == name:
            return m
    return None

In [13]:
out_layer = get_named_module_from_model(model, "1.4")

In [32]:
img_vecs = {}

with Hook(out_layer, get_output, True, True) as hook:
    start = time.time()
    for i, (xb, yb) in enumerate(dataloader):
        bs = xb.shape[0]
        img_ids = dataloader.items[i*bs: (i+1)*bs]
        result = model.eval()(xb)
        img_reprs = hook.stored.cpu().numpy()
        print(img_reprs)
        img_reprs = img_reprs.reshape(bs, -1)
        for img_id, img_repr in zip(img_ids, img_reprs):
            img_repr_map[img_id] = img_repr

[[0.       0.       0.       0.       ... 0.970437 1.730672 1.717203 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       1.695651 0.      ]
 [0.       0.       0.       0.       ... 1.868422 0.       3.769852 0.      ]
 [0.       0.       0.       0.       ... 1.085263 0.       1.282947 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.254855 0.78448  1.682872 0.      ]
 [0.275125 0.       0.       0.       ... 0.       2.156593 4.049745 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       4.236913 0.      ]
 [0.       0.       0.       0.       ... 0.07265  0.815412 3.332256 0.      ]]
[[0.       0.       0.       0.       ... 1.075221 2.22124  3.46255  0.      ]
 [0.       0.       0.       0.       ... 0.552649 0.689899 3.265951 0.      ]
 [0.       0.       0.       0.851781 ... 0.       2.398453 4.750698 0.      ]
 [0.       0.       0.491939 1.586208 ... 3.271238 0.       4.259107 0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.       ... 0.       0.922217 2.184268 0.      ]
 [0.       0.       0.       0.       ... 2.599903 0.       2.286766 0.      ]
 [0.       0.       0.       0.       ... 1.484385 0.       2.320165 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       3.34348  0.      ]
 ...
 [0.       0.       0.       0.       ... 0.959192 1.838321 4.725713 0.      ]
 [0.       0.       0.       1.055732 ... 0.       1.661766 2.602375 0.      ]
 [0.       0.       0.       0.504984 ... 2.812556 0.559874 5.327992 0.      ]
 [0.       0.       0.       0.       ... 0.       0.604874 4.514921 0.      ]]
[[0.       0.       0.       0.       ... 0.451633 2.129893 5.8126   0.      ]
 [0.       0.       0.010442 0.       ... 0.       0.       2.955616 0.      ]
 [0.       0.       0.       0.       ... 0.356734 0.32218  3.153028 0.      ]
 [0.       0.       0.       0.       ... 0.029582 0.324316 3.761045 0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.44376  ... 1.426862 0.       2.463756 0.      ]
 [0.       0.       0.       0.       ... 0.743965 0.840572 1.18986  0.      ]
 [0.       0.       0.       0.       ... 0.21757  1.040758 2.38822  0.      ]
 [0.       0.       0.       0.       ... 0.443917 1.787766 2.844798 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.715864 0.276773 0.847882 0.      ]
 [0.       0.       0.       0.       ... 1.261164 0.       4.637238 0.      ]
 [0.       0.       0.       0.       ... 1.867413 0.52096  0.080795 0.      ]
 [0.       0.       0.       0.       ... 1.398966 0.126944 1.640263 0.      ]]
[[0.       0.       0.       0.       ... 0.441981 0.       2.416637 0.      ]
 [0.       0.       0.       0.       ... 1.27905  1.019815 1.811253 0.      ]
 [0.       0.       0.       0.       ... 0.405544 1.778719 3.93022  0.      ]
 [0.       0.       0.       0.       ... 0.109356 0.       0.       0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.       ... 1.393227 1.123078 4.123558 0.      ]
 [0.       0.       0.       0.       ... 2.386974 0.308445 4.257157 0.      ]
 [0.       0.       0.       0.       ... 1.909734 0.542668 1.964134 0.      ]
 [0.       0.       0.       0.       ... 0.492091 1.582109 2.807994 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.829901 0.       1.303923 0.      ]
 [0.       0.       0.       0.       ... 0.       0.400806 2.147327 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       0.       0.      ]
 [0.       0.       0.       0.       ... 1.600439 0.       1.16254  0.      ]]
[[0.       0.       0.       0.       ... 0.       2.383992 3.779638 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       2.228703 0.      ]
 [0.       0.       0.       0.       ... 0.693284 0.608203 1.193192 0.      ]
 [0.       0.075015 0.       0.       ... 1.505881 0.       1.769691 0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.       ... 0.296485 0.       2.235747 0.      ]
 [0.       0.       0.       0.       ... 1.710504 0.777153 2.13129  0.      ]
 [0.       0.       0.       0.       ... 2.723057 0.       1.430356 0.      ]
 [0.       0.       0.       0.       ... 0.891449 0.       2.268249 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.551935 0.508494 1.411396 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       3.048881 0.      ]
 [0.       0.       0.       0.       ... 0.373673 0.660143 2.862944 0.      ]
 [0.       0.       0.       0.       ... 0.       0.466825 3.862116 0.      ]]
[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 ... 1.106488e+00 3.889148e-01 3.015391e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 ... 1.910463e+00 7.795215e-02 1.758587e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 ... 3.775835e-03 0.000000e+00 1.626261e+00 0.000000e+00]
 [0.000000e+00 3.895307e-02 0

...

[[0.       0.       0.       0.       ... 1.053829 0.       0.81962  0.      ]
 [0.       0.       0.       0.       ... 1.969337 0.       0.93675  0.      ]
 [0.       0.       0.       0.       ... 0.       0.614245 3.822706 0.      ]
 [0.       0.       0.       0.       ... 1.330209 0.853166 2.041119 0.      ]
 ...
 [0.       0.       0.       0.       ... 1.39439  0.761447 5.230814 0.      ]
 [0.       0.       0.       0.       ... 2.258284 1.44069  2.035291 0.      ]
 [0.       0.       0.       0.       ... 1.296659 0.038689 3.786339 0.      ]
 [0.       0.       0.       0.       ... 0.       1.693019 4.114963 0.      ]]
[[0.       0.       0.090428 0.       ... 0.       0.       1.309133 0.      ]
 [0.       0.       0.       0.       ... 2.069835 0.       3.816656 0.      ]
 [0.       0.       1.182982 0.       ... 1.777665 0.       3.412388 0.      ]
 [0.       0.       0.       0.       ... 0.086284 0.       2.63419  0.      ]
 ...
 [0.       0.       0.935385 0.       ...

[[0.       0.       0.       0.       ... 0.778271 0.771928 1.908168 0.      ]
 [0.       0.       0.       0.       ... 1.058267 1.813798 1.007641 0.      ]
 [0.       0.       0.       0.       ... 0.189103 0.640406 0.       0.      ]
 [0.       0.       0.349197 0.       ... 1.306529 0.034484 3.892713 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.       0.928923 3.156554 0.      ]
 [0.       0.       0.       0.       ... 1.524794 0.600051 2.169755 0.      ]
 [0.       0.       0.       0.       ... 2.204138 0.858473 1.848819 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       3.536864 0.      ]]
[[0.       0.       0.       0.726197 ... 1.224874 0.       4.000624 0.      ]
 [0.       0.       0.       0.       ... 1.342352 0.       0.630689 0.      ]
 [0.       0.       0.       0.       ... 1.102929 0.       1.102798 0.      ]
 [0.       0.       0.       0.486458 ... 2.453665 0.       1.564339 0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.       ... 0.959124 0.425499 3.744258 0.      ]
 [0.       0.       0.       0.       ... 2.201038 0.419982 2.238395 0.      ]
 [0.       0.       0.       0.       ... 1.086936 0.       2.11471  0.      ]
 [0.       0.       0.       0.       ... 0.       1.858392 3.377299 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.217539 0.689918 1.646003 0.      ]
 [0.       0.       0.       0.       ... 0.929118 0.       1.455974 0.      ]
 [0.       0.       0.       0.       ... 1.338528 1.178268 2.884228 0.      ]
 [0.       0.       0.       0.       ... 0.602439 0.372005 2.359123 0.      ]]
[[0.       0.       0.       0.       ... 1.010764 0.       0.991424 0.      ]
 [0.       0.       0.       0.       ... 1.233499 0.       4.516732 0.      ]
 [0.       0.       0.55633  0.28795  ... 0.       2.007897 6.795907 0.      ]
 [0.       0.       0.       0.       ... 0.81266  0.382278 1.844539 0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.       ... 1.287576 1.481248 5.791582 0.      ]
 [0.       0.       0.       0.       ... 0.444488 0.802386 1.654164 0.      ]
 [0.       0.       0.       0.       ... 1.805666 0.943473 1.200599 0.      ]
 [0.       0.       0.       0.       ... 0.       0.318388 2.486302 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.       0.       0.310067 0.      ]
 [0.       0.       0.       0.       ... 0.621879 0.644819 1.660804 0.      ]
 [0.       0.       0.       0.       ... 2.51728  1.986589 3.393995 0.      ]
 [0.       0.       0.       0.       ... 0.2886   0.       3.105321 0.      ]]
[[0.       0.       0.       0.       ... 1.145242 1.854213 3.760531 0.      ]
 [0.       0.       1.298624 0.       ... 2.043348 0.32264  4.743244 0.      ]
 [0.       0.       0.       0.       ... 1.341541 0.       2.801053 0.      ]
 [0.       0.       0.       0.       ... 1.101679 0.085507 1.391152 0.      ]
 ...
 [0.       0.       0.       0.       ...

[[0.       0.       0.       0.       ... 1.858243 1.192365 3.540623 0.      ]
 [0.       0.       0.       0.       ... 1.957129 0.162734 3.632126 0.      ]
 [0.       0.       0.       0.       ... 0.450612 0.919913 2.89107  0.      ]
 [0.       0.       0.       0.       ... 0.       2.569505 5.513924 0.      ]
 ...
 [0.       0.       0.       0.       ... 0.744169 0.924275 2.243245 0.      ]
 [0.       0.       0.       0.       ... 0.       0.       2.412526 0.      ]
 [0.       0.       0.670705 0.       ... 0.       0.       4.61536  0.      ]
 [0.       0.       0.       0.       ... 1.867355 1.469337 3.113851 0.      ]]
[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 ... 1.081680e+00 8.570337e-01 1.876792e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 ... 6.097931e-01 0.000000e+00 4.004129e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00 1.696113e+00 0.000000e+00]
 [2.511263e-03 0.000000e+00 0

In [69]:
df = pd.DataFrame(img_vecs.items(), columns=['img_id', 'img_vec'])

In [70]:
df.head()

,img_id,img_vec
0,/home/sloth/Desktop/animeface/animeface-2009/a...,"[0.0, 0.0, 0.0, 0.0, 0.06119205057621002, 2.48..."
1,/home/sloth/Desktop/animeface/animeface-2009/a...,"[0.0, 0.0, 0.0, 0.0, 0.7806511521339417, 1.550..."
2,/home/sloth/Desktop/animeface/animeface-2009/a...,"[0.0, 0.0, 0.0, 0.0, 0.0, 3.345784902572632, 0..."
3,/home/sloth/Desktop/animeface/animeface-2009/a...,"[0.0, 0.0, 0.0, 0.0, 0.44981351494789124, 1.80..."
4,/home/sloth/Desktop/animeface/animeface-2009/a...,"[0.0, 0.0, 0.0, 0.08110424876213074, 0.5474264..."


In [71]:
df.to_csv("img_feature_vecs_3.csv",index=False)